# Lab Streaming Layer (LSL) tutorial for EEG data collection and synchronization using EMOTIV software

Welcome! In this tutorial we'll learn how to use Lab Streaming Layer (LSL) in Python to collect and synchronize Emotiv EEG data from multiple devices. It will require a basic working knowledge of Python programming language. We will learn:
1. What is Lab Streaming Layer and why we would use it
2. How to collect synchronized data from multiple Emotiv EEG devices
3. How to import and look through the data

## What is LSL and what is it good for?
Increasingly capable, precise and mobile brain- and body-sensing hardware devices (like EMOTIV EEG systems) are bringing neuroscience outside the lab into the world of real-time data. Where brain measurements like EEG and MEG had once been confined to research labs, mobile devices let us collect multiple of data in more naturalistic environments, and from multiple people at once. A researcher may be interested in physiological synchrony between two people listening to the same music. LSL can help us collect data from two EEG headsets separately that is also synchronized to the presentation of sound.

- LSL is an open-source networked middleware ecosystem to stream, receive, synchronize, and record neural, physiological, and behavioral data streams acquired from diverse sensor hardware.” - https://labstreaminglayer.org/#/

Some examples of other uses for LSL:
1. Add event markers from an experiment to an ongoing EEG data
2. Time-align data from multiple sources for a single participant (e.g. heart rate, EMG, EEG)
3. Time-align data from multiple participants (e.g. EEG hyperscanning)

## How does it work?
Lab Streaming Layer is a protocol for the real-time exchange of time-series data between multiple devices. LSL can be implemented using open-source libraries for programming languages like Python, MATLAB, C++, Java and others. 

The core functionality revolves around LSL **data streams**:
1. **An acquisition device/software collects data and creates a data stream**
    - Physiological data can be streamed to LSL from EEG recording devices, eye-trackers, motion capture systems, heart rate monitors, etc., including metadata (sampling rate, data type, channel information, etc.)
    - Event markers from experiments (e.g. using PsychoPy) can also be sent as a data stream using LSL
2. **The data stream is *published* to the network**
    - This is how data is *sent* using LSL; the data stream is "broadcast" to the network
    - Published streams are available on the network and discoverable by other LSL-supported devices on the same network
    - LSL assigns each data chunk or sample a timestamp based on a *common clock* (following the *Network Time Protocol*). 
    - The stream is pushed sample-by-sample (or chunk-by-chunk) through an "outlet"
3. **Collection device(s) “subscribe” to data stream(s)**
    - This is how data is *received* using LSL
    - Collection devices on the same network receive published data streams via "inlets".
    - Each inlet receives the stream samples and metadata from only one outlet
4. **Save data**
    - Upon subscribing to a data stream, you can save it to a variable in your preferred programming language, or use LSL's provided software LabRecorder to save it to a standard format such as .xdf.

# Tutorial overview 
In this tutorial, we'll take an example experimental setup and guide you through the necessary steps and code for implementing it using LSL in Python.

We'll use a Python play a sound while collecting EEG data from two people wearing Emotiv headsets. We'll use two computers each running EmotivPRO to collect the EEG data, and broadcast each stream through a separate LSL outlet. We'll use a Python library to play an audio file and simultaneously send a trigger each time the file starts.

STEPS:
1. Use EmotivPRO to **create and publish data streams through *LSL outlets*** that includes EEG data (and/or motion, contact quality, signal quality, etc.)
2. Play audio using the sound library `soundfile`, and simultaneously send a trigger through another *LSL outlet*
3. Use LabRecorder to capture and save all three data streams through an *LSL inlet*

For more details, see the resources at the bottom of this tutorial.

---
## STEP 1 - Setup and install
1. Install EmotivPRO on your device(s). You will need a valid Emotiv license.
2. Install the [Python LSL library](https://github.com/labstreaminglayer) with the following command:
    
    `pip install pylsl`
3. Download the [LabRecorder software](https://github.com/labstreaminglayer/App-LabRecorder/releases) software. The is a simple, free app that can be run from the command line or using a standalone download

For our experiment

4. Install the necessary packages for playing audio using Python
    
    `pip install sounddevice soundfile`

---
## STEP 2 - Set up the EEG devices

1. Fit your Emotiv device(s) on your participant(s), turn them on, and connect them to your computer(s) via Bluetooth
2. Open EmotivPRO and ensure the EEG data quality is sufficient using the sensor checks
    - EEG data is now being shown in the application

---
## STEP 3 - Send the data from EmotivPRO via an LSL stream
1. Locate the "..." in the upper right corner of the app, navigate to Settings
2. Find the 'Lab Streaming Layer' section and the 'Outlet' subsection
3. Select all the datatypes that you would like to broadcast
4. Select the data format (32-bit float or 64-bit double)
5. Select whether to send data sample-by-sample or in chunks of samples
6. Click 'Start' to broadcast an LSL data stream


---
## STEP 4 - Use a Python script to play audio and send triggers
1. Copy and paste the following codeblock into a python file and save it to your computer
2. Locate an audio file (ideally a .wav file) you'd like to play and edit the script by changing the variable `audio_filepath` to the filepath to your audio file on your computer
3. Open a command prompt to interact with the command line and navigate to the folder where your Python file is stored
4. Enter: `python3 filename.py`
    - Depending your Python install, you may use `python` instead of `python3`

In [ ]:
"""LSL example - playing audio and sending a trigger
This script shows minmimal example code that will allow a user to play and audio file and simultaneously send a a trigger through an LSL stream that can be captured (for instance, using LabRecorder software) and synchronized with other LSL data streams.

Operation:
Once run in the Terminal, this script immediately initates an LSL stream. Whenever the 'Enter' key is pressed, it sends a trigger and plays an audio file."""

import sounddevice as sd
import soundfile as sf
from pylsl import StreamInfo, StreamOutlet

def wait_for_keypress():
    print("Press ENTER to start audio playback and send an LSL marker.")

    while True: # This loop waits for a keyboard input
        input_str = input()  # Wait for input from the terminal
        if input_str == "":  # If the enter key is pressed, proceed
            break

def AudioMarker(audio_file, outlet): # function for playing audio and sending marker
    data, fs = sf.read(audio_file) # Load the audio file

    print("Playing audio and sending LSL marker...")
    marker_val = [1]
    outlet.push_sample(marker_val) # Send marker indicating the start of audio playback
    sd.play(data, fs) # play the audio
    sd.wait()  # Wait until audio is done playing
    print("Audio playback finished.")

if __name__ == "__main__": # MAIN LOOP
    # Setup LSL stream for markers
    stream_name = 'AudioMarkers'
    stream_type = 'Markers'
    n_chans = 1
    sr = 0  # Set to 0 sampling rate because markers are irregular
    chan_format = 'int32'
    marker_id = 'uniqueMarkerID12345'

    info = StreamInfo(stream_name, stream_type, n_chans, sr, chan_format, marker_id)
    outlet = StreamOutlet(info) # create LSL outlet

    # Keep the script running and wait for ENTER key to play audio and send marker
    while True:
        wait_for_keypress()
        audio_filepath = "/path/to/your/audio_file.wav"  # replace with correct path to your audio file
        AudioMarker(audio_filepath, outlet)
        # After playing audio and sending a marker, the script goes back to waiting for the next keypress

---
***By running this file (even before playing the audio), you've initiated an LSL stream through an *outlet*. Now we'll view that stream in LabRecorder

---

## STEP 5 - Use LabRecorder to view and save all LSL streams
1. Open LabRecorder
2. Press `Update`. The available LSL streams should be visible in the stream list
    - You should be able to see streams from both EmotivPROs (usually called "EmotivDataStream") and the marker stream (called "AudioMarkers")
3. Click `Browse` to select a location to store data (and set other parameters)
4. Select all streams and press `Record` to start recording

---
# Working with the data
LabRecorder outputs an XDF file (Extensible Data Format) that contains data from all the streams. XDF files are structured into, *streams*, each with a different *header* that describes what it contains (device name, data type, sampling rate, channels, and more). You can use the codeblock to open your XDF file and display some basic information.

In [ ]:
import pyxdf
import mne
import matplotlib.pyplot as plt
import numpy as np

# Path to your XDF file
data_path = '/Users/lucas/Desktop/Emotiv/LSL/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf'

# Load the XDF file
streams, fileheader = pyxdf.load_xdf(data_path)
print("XDF File Header:", fileheader)
print("Number of streams found:", len(streams))

for i, stream in enumerate(streams):
    print("\nStream", i + 1)
    print("Stream Name:", stream['info']['name'][0])
    print("Stream Type:", stream['info']['type'][0])
    print("Number of Channels:", stream['info']['channel_count'][0])
    sfreq = float(stream['info']['nominal_srate'][0])
    print("Sampling Rate:", sfreq)
    print("Number of Samples:", len(stream['time_series']))
    print("Print the first 5 data points:", stream['time_series'][:5])

    channel_names = [chan['label'][0] for chan in stream['info']['desc'][0]['channels'][0]['channel']]
    print("Channel Names:", channel_names)
    channel_types = 'eeg'




---
## Analyzing the data
[MNE](https://mne.tools/stable/index.html) is a powerful library for analyzing time series data like EEG. How best to preprocess, display and anayze your EEG data depends on your what you intend to study. Below is a codeblock that you can use to create a `raw` object in MNE and view a spectrogram of your data.

In [ ]:
# Create MNE info object
info = mne.create_info(channel_names, sfreq, channel_types)
data = np.array(stream['time_series']).T # Data needs to be transposed: channels x samples
raw = mne.io.RawArray(data, info)
raw.plot_psd(fmax=50) # plot a simple spectrogram (power spectral density)

---
## Things to consider with Hyperscanning studies
1. Network bandwidth:
    - LSL works via a network. Sending multiple streams of large data can overload a local wireless network and delay latencies. When possible, choose a wired (ethernet) connection. 
2. Keep settings consisten:
    - Try to keep device settings as consistent as possible between participants (e.g. sampling rate)




---
# Additional resources

1. Check out the [online documentation](https://labstreaminglayer.readthedocs.io/index.html), including the official [README file on GitHub](https://github.com/sccn/labstreaminglayer/)
2. You'll need one or more [supported data acquisition device](https://labstreaminglayer.readthedocs.io/info/supported_devices.html) for collecting data
    - All of EMOTIV's brainware devices connect to EmotivPRO software, which has LSL built-in capabilities for sending and recieving data streams
3. Additional resources:
    - [Code](https://github.com/Emotiv/labstreaminglayer) to run LSL using Emotiv’s devices, with example scripts
    - Useful LSL [demo on YouTube](https://www.youtube.com/watch?v=Y1at7yrcFW0&list=PLVnr33MP5RMRhGwY36zHHDOmAaYTB138D)
    - [SCCN LSL GitHub repository](https://github.com/sccn/labstreaminglayer) for all associated libraries
    - [LSL GitHub repository](https://github.com/labstreaminglayer) for a collection a submodules and apps
4. [HyPyP analysis pipeline](https://academic.oup.com/scan/article/16/1-2/72/5919711?login=false) for Hyperscanning studies